In [ ]:
import altair as alt
import polars as pl

In [ ]:
years = range(2021, 2026, 2)
filepaths = {
    y: rf"\\Store\gis\Data\Observed\Streets\Speed\CMP-floating_car_run\{y}\floating_car-speed-summary_stats.csv"
    for y in years
}
dfs = {y: pl.read_csv(filepaths[y]) for y in years}

In [ ]:
comparison_year = 2025
base_year = 2023

In [ ]:
wide_index_cols = ["cmp_segid", "period", "direction"]


def make_wide_df(dfs, years, index_cols):
    speed_cols = ["avg_speed", "std_speed", "min_speed", "max_speed"]
    df = dfs[years[0]].select(index_cols)
    for y in years:
        df = df.join(
            dfs[y].select(
                *index_cols,
                pl.col(speed_cols).name.suffix(f"-{y}"),
                (pl.col("std_speed") / pl.col("avg_speed")).alias(f"std/avg_speed-{y}"),
            ),
            on=index_cols,
        )
    return df


def make_long_df(dfs, years):
    return pl.concat(
        (dfs[y].with_columns(pl.lit(y).alias("year")) for y in years)
    ).with_columns((pl.col("std_speed") / pl.col("avg_speed")).alias("std/avg_speed"))


def calc_diff(wide_df, comparison_year, base_year):
    diff_col = f"avg_speed-diff-{comparison_year}-{base_year}"
    diff_pct_col = f"avg_speed-diff-pct-{comparison_year}-{base_year}"
    return wide_df.with_columns(
        (
            pl.col(f"avg_speed-{comparison_year}") - pl.col(f"avg_speed-{base_year}")
        ).alias(diff_col)
    ).select(
        *wide_index_cols,
        diff_col,
        (pl.col(diff_col) / pl.col("avg_speed-2021") * 100).alias(diff_pct_col),
    )

In [ ]:
wide_df = make_wide_df(dfs, years, wide_index_cols)
long_df = make_long_df(dfs, years)

In [ ]:
alt.Chart(long_df).mark_bar().encode(
    x=alt.X("avg_speed-binned:O"), y="count()", color="year:N", xOffset="year:N"
).transform_bin("avg_speed-binned", field="avg_speed")

In [ ]:
# there's an outlier in the 60s in 2021
alt.Chart(long_df).mark_bar(opacity=0.3).encode(
    x=alt.X("std_speed-binned:Q"),
    y=alt.Y("count()").stack(None),
    color="year:N",
).transform_bin("std_speed-binned", field="std_speed", bin=alt.BinParams(step=2))

In [ ]:
# there's an outlier in the 60s in 2021
alt.Chart(long_df.filter(pl.col("std_speed") < 60)).mark_bar().encode(
    x=alt.X("std_speed-binned:O"), y=alt.Y("count()"), color="year:N", xOffset="year:N"
).transform_bin("std_speed-binned", field="std_speed", bin=alt.BinParams(step=2))

In [ ]:
alt.Chart(long_df).mark_bar().encode(
    x=alt.X("std/avg_speed-binned:O"), y="count()", color="year:N", xOffset="year:N"
).transform_bin("std/avg_speed-binned", field="std/avg_speed")

In [ ]:
diff_df = calc_diff(wide_df, 2025, 2023)

alt.Chart(diff_df).mark_bar().encode(
    x=alt.X(f"avg_speed-diff-{comparison_year}-{base_year}").bin(step=2.5),
    y="count()",
) | alt.Chart(diff_df).mark_bar().encode(
    x=alt.X(f"avg_speed-diff-pct-{comparison_year}-{base_year}").bin(step=20),
    y="count()",
)

In [ ]:
alt.Chart(wide_df).mark_circle().encode(
    x=f"avg_speed-{base_year}",
    y=f"avg_speed-{comparison_year}",
    color="period",
    tooltip=[
        "cmp_segid",
        "direction",
        "period",
        f"avg_speed-{base_year}",
        f"avg_speed-{comparison_year}",
    ],
).interactive()

In [ ]:
wide_df.join(diff_df, on=wide_index_cols).sort(
    f"avg_speed-diff-pct-{comparison_year}-{base_year}"
)

In [ ]:
wide_df.join(diff_df, on=wide_index_cols).sort(
    f"std/avg_speed-{comparison_year}", descending=True
).head(10)